In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [69]:
#use requests

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

response = requests.get('https://www.wordstream.com/blog/conversion-rate-benchmarks', headers = headers)


## 1st Option Without BeautifulSoup

In [169]:
table = pd.read_html(response.text)

google_ads_cvr = (table[0])
facebook_ads_cvr = (table[2])

/tmp/ipython-input-3906586548.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(response.text)


In [170]:
# make indexis
google_ads_cvr = google_ads_cvr.rename(columns={0: 'Industry', 1: 'Benchmark_CVR'})
google_ads_cvr = google_ads_cvr.iloc[1:].reset_index(drop=True)

desired_industries = [
    'Beauty & Personal Care',
    'Career & Employment',
    'Health & Fitness',
    'Restaurants & Food',
    'Sports & Recreation'
]

google_ads_cvr = google_ads_cvr[google_ads_cvr['Industry'].isin(desired_industries)].reset_index(drop=True)

google_ads_cvr['Benchmark_CVR'] = google_ads_cvr['Benchmark_CVR'].astype(str).str.replace('%','').astype(float) / 100

google_ads_cvr['Industry'] = google_ads_cvr['Industry'].replace({
    'Beauty & Personal Care': 'Fashionistas',
    'Career & Employment': 'Tech Enthusiasts',
    'Health & Fitness': 'Health & Wellness',
    'Restaurants & Food': 'Foodies',
    'Sports & Recreation': 'Outdoor Adventurers'
})

google_ads_cvr['Join_Key'] =  'Google Ads_' + google_ads_cvr['Industry']
google_ads_cvr

,Industry,Benchmark_CVR,Join_Key
0,Fashionistas,0.0782,Google Ads_Fashionistas
1,Tech Enthusiasts,0.0433,Google Ads_Tech Enthusiasts
2,Health & Wellness,0.0680,Google Ads_Health & Wellness
3,Foodies,0.0709,Google Ads_Foodies
4,Outdoor Adventurers,0.0762,Google Ads_Outdoor Adventurers


In [171]:
facebook_ads_cvr = facebook_ads_cvr.rename(columns={0: 'Industry', 1: 'Benchmark_CVR'})
facebook_ads_cvr = facebook_ads_cvr.iloc[1:].reset_index(drop=True)


facebook_ads_cvr = facebook_ads_cvr[facebook_ads_cvr['Industry'].isin(desired_industries)].reset_index(drop=True)

facebook_ads_cvr['Benchmark_CVR'] = facebook_ads_cvr['Benchmark_CVR'].astype(str).str.replace('%','').astype(float) / 100

facebook_ads_cvr['Industry'] = facebook_ads_cvr['Industry'].replace({
    'Beauty & Personal Care': 'Fashionistas',
    'Career & Employment': 'Tech Enthusiasts',
    'Health & Fitness': 'Health & Wellness',
    'Restaurants & Food': 'Foodies',
    'Sports & Recreation': 'Outdoor Adventurers'
})

facebook_ads_cvr['Join_Key'] =  'Facebook_' + facebook_ads_cvr['Industry']
facebook_ads_cvr

,Industry,Benchmark_CVR,Join_Key
0,Fashionistas,0.0529,Facebook_Fashionistas
1,Tech Enthusiasts,0.0577,Facebook_Tech Enthusiasts
2,Health & Wellness,0.0563,Facebook_Health & Wellness
3,Foodies,0.1825,Facebook_Foodies
4,Outdoor Adventurers,0.0548,Facebook_Outdoor Adventurers


In [51]:
mc_df = pd.read_csv('/content/marketing_campaign_dataset.csv')

In [114]:
mc_df['Join_Key'] = mc_df['Channel_Used'] + '_' + mc_df['Customer_Segment']

In [115]:
benchmarks = mc_df.groupby('Join_Key')['Conversion_Rate'].mean().round(3)

In [116]:
benchmarks

,Conversion_Rate
Join_Key,
Email_Fashionistas,0.080
Email_Foodies,0.080
Email_Health & Wellness,0.080
Email_Outdoor Adventurers,0.081
Email_Tech Enthusiasts,0.080
Facebook_Fashionistas,0.080
Facebook_Foodies,0.080
Facebook_Health & Wellness,0.080
Facebook_Outdoor Adventurers,0.080


In [172]:
merge_google = pd.merge(benchmarks, google_ads_cvr, on='Join_Key', how='outer')
merge_benchmarkes = pd.merge(merge_google, facebook_ads_cvr, on='Join_Key', how='outer')

In [173]:
from pandas.core.reshape import merge
merge_benchmarkes['Conversion_Rate'] = merge_benchmarkes['Benchmark_CVR_x'].fillna(merge_benchmarkes['Conversion_Rate'])
merge_benchmarkes['Conversion_Rate'] = merge_benchmarkes['Benchmark_CVR_y'].fillna(merge_benchmarkes['Conversion_Rate'])
merge_benchmarkes = merge_benchmarkes.drop(columns=['Industry_x',	'Benchmark_CVR_x',	'Industry_y',	'Benchmark_CVR_y'])

In [174]:
merge_benchmarkes

,Join_Key,Conversion_Rate
0,Email_Fashionistas,0.0800
1,Email_Foodies,0.0800
2,Email_Health & Wellness,0.0800
3,Email_Outdoor Adventurers,0.0810
4,Email_Tech Enthusiasts,0.0800
5,Facebook_Fashionistas,0.0529
6,Facebook_Foodies,0.1825
7,Facebook_Health & Wellness,0.0563
8,Facebook_Outdoor Adventurers,0.0548
9,Facebook_Tech Enthusiasts,0.0577


In [175]:
mc_df_new = pd.merge(mc_df, merge_benchmarkes, on='Join_Key', how='left')
mc_df_new = mc_df_new.drop(columns=['Key_ChanelSegment',	'Benchmark', 'Key_Chanel_Segment'])

In [177]:
mc_df_new = mc_df_new.rename(columns={'Conversion_Rate_x': 'Conversion_Rate', 'Conversion_Rate_y': 'Benchmark'})
mc_df_new['Benchmark'] = mc_df_new['Benchmark'].round(2)
mc_df_new.head(15)

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Join_Key,Benchmark
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,"$16,174.00",6.29,Chicago,Spanish,506,1922,6,Health & Wellness,1/1/2021,Google Ads_Health & Wellness,0.07
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,"$11,566.00",5.61,New York,German,116,7523,7,Fashionistas,1/2/2021,Google Ads_Fashionistas,0.08
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,"$10,200.00",7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,1/3/2021,YouTube_Outdoor Adventurers,0.08
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,"$12,724.00",5.55,Miami,Mandarin,217,1820,7,Health & Wellness,1/4/2021,YouTube_Health & Wellness,0.08
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,"$16,452.00",6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,1/5/2021,YouTube_Health & Wellness,0.08
5,6,DataTech Solutions,Display,All Ages,15 days,Instagram,0.07,"$9,716.00",4.36,New York,German,100,1643,1,Foodies,1/6/2021,Instagram_Foodies,0.08
6,7,NexGen Systems,Email,Women 35-44,60 days,Website,0.13,"$11,067.00",2.86,Los Angeles,Spanish,817,8749,10,Tech Enthusiasts,1/7/2021,Website_Tech Enthusiasts,0.08
7,8,DataTech Solutions,Search,Men 18-24,45 days,Google Ads,0.08,"$13,280.00",5.55,Los Angeles,Mandarin,624,7854,7,Outdoor Adventurers,1/8/2021,Google Ads_Outdoor Adventurers,0.08
8,9,Alpha Innovations,Social Media,Women 35-44,15 days,Facebook,0.09,"$18,066.00",6.73,Chicago,German,861,1754,6,Tech Enthusiasts,1/9/2021,Facebook_Tech Enthusiasts,0.06
9,10,TechCorp,Email,Women 35-44,15 days,Instagram,0.09,"$13,766.00",3.78,Los Angeles,English,642,3856,3,Tech Enthusiasts,1/10/2021,Instagram_Tech Enthusiasts,0.08


## 2nd Option With BeautifulSoup

In [182]:
soup = BeautifulSoup(response.text, 'html.parser')

categories = soup.find_all('table')

all_data = []

#we want to get all cvr data for google ads and facebook ads in one table
for table in categories:
    prev_header = table.find_previous(['h1', 'h2', 'h3'])
    table_context = prev_header.text.lower() if prev_header else ""

    # filter the needed tables
    if 'conversion rate' not in table_context:
        continue

    # deefine sources for our table, it would be google or facebook
    if 'google' in table_context:
        sourse = 'Google Ads'
    elif 'facebook' in table_context:
        sourse = 'Facebook'
    else:
        continue

    rows = table.find_all('tr') # tr is the whole row
    for row in rows:
        cols = row.find_all('td') # td is whole columns


        if len(cols) >= 2:
            industry = cols[0].text.strip() # the first column
            cvr = cols[1].text.strip() # the second column

            if industry.lower() in ['business category', 'average cvr']: # these words shouldn`t be in our table
                continue

            all_data.append({
                'Sourse': sourse,
                'Industry': industry,
                'Benchmark_CVR': cvr
            })

In [183]:
# create a dataframe
df_cvr = pd.DataFrame(all_data)
df_cvr['Benchmark_CVR'] = df_cvr['Benchmark_CVR'].str.replace('%', '').astype(float) / 100
df_cvr.head()

,Sourse,Industry,Benchmark_CVR
0,Google Ads,Animals & Pets,0.1307
1,Google Ads,Apparel / Fashion & Jewelry,0.0399
2,Google Ads,Arts & Entertainment,0.0484
3,Google Ads,Attorneys & Legal Services,0.0509
4,Google Ads,Automotive — For Sale,0.0776


In [184]:
# desired_industries = [
#     'Beauty & Personal Care',
#     'Career & Employment',
#     'Health & Fitness',
#     'Restaurants & Food',
#     'Sports & Recreation'
# ]

df_cvr = df_cvr[df_cvr['Industry'].isin(desired_industries)].reset_index(drop=True)

df_cvr['Industry'] = df_cvr['Industry'].replace({
    'Beauty & Personal Care': 'Fashionistas',
    'Career & Employment': 'Tech Enthusiasts',
    'Health & Fitness': 'Health & Wellness',
    'Restaurants & Food': 'Foodies',
    'Sports & Recreation': 'Outdoor Adventurers'
})

df_cvr['Join_Key'] =  df_cvr['Sourse'] + '_' + df_cvr['Industry']
df_cvr

,Sourse,Industry,Benchmark_CVR,Join_Key
0,Google Ads,Fashionistas,0.0782,Google Ads_Fashionistas
1,Google Ads,Tech Enthusiasts,0.0433,Google Ads_Tech Enthusiasts
2,Google Ads,Health & Wellness,0.0680,Google Ads_Health & Wellness
3,Google Ads,Foodies,0.0709,Google Ads_Foodies
4,Google Ads,Outdoor Adventurers,0.0762,Google Ads_Outdoor Adventurers
5,Facebook,Fashionistas,0.0529,Facebook_Fashionistas
6,Facebook,Tech Enthusiasts,0.0577,Facebook_Tech Enthusiasts
7,Facebook,Health & Wellness,0.0563,Facebook_Health & Wellness
8,Facebook,Foodies,0.1825,Facebook_Foodies
9,Facebook,Outdoor Adventurers,0.0548,Facebook_Outdoor Adventurers


In [186]:
merge_benchmarkes2 = pd.merge(benchmarks, df_cvr, on='Join_Key', how='outer')
# from pandas.core.reshape import merge
merge_benchmarkes2['Conversion_Rate'] = merge_benchmarkes2['Benchmark_CVR'].fillna(merge_benchmarkes2['Conversion_Rate'])
merge_benchmarkes2 = merge_benchmarkes2.drop(columns=['Sourse',	'Industry',	'Benchmark_CVR'])
merge_benchmarkes2

,Join_Key,Conversion_Rate
0,Email_Fashionistas,0.0800
1,Email_Foodies,0.0800
2,Email_Health & Wellness,0.0800
3,Email_Outdoor Adventurers,0.0810
4,Email_Tech Enthusiasts,0.0800
5,Facebook_Fashionistas,0.0529
6,Facebook_Foodies,0.1825
7,Facebook_Health & Wellness,0.0563
8,Facebook_Outdoor Adventurers,0.0548
9,Facebook_Tech Enthusiasts,0.0577


In [188]:
mc_df_new2 = pd.merge(mc_df, merge_benchmarkes2, on='Join_Key', how='left')
mc_df_new2 = mc_df_new2.drop(columns=['Key_ChanelSegment',	'Benchmark', 'Key_Chanel_Segment'])
mc_df_new2 = mc_df_new2.rename(columns={'Conversion_Rate_x': 'Conversion_Rate', 'Conversion_Rate_y': 'Benchmark'})
mc_df_new2['Benchmark'] = mc_df_new2['Benchmark'].round(2)
mc_df_new2.head(15)

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,Join_Key,Benchmark
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,"$16,174.00",6.29,Chicago,Spanish,506,1922,6,Health & Wellness,1/1/2021,Google Ads_Health & Wellness,0.07
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,"$11,566.00",5.61,New York,German,116,7523,7,Fashionistas,1/2/2021,Google Ads_Fashionistas,0.08
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,"$10,200.00",7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,1/3/2021,YouTube_Outdoor Adventurers,0.08
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,"$12,724.00",5.55,Miami,Mandarin,217,1820,7,Health & Wellness,1/4/2021,YouTube_Health & Wellness,0.08
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,"$16,452.00",6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,1/5/2021,YouTube_Health & Wellness,0.08
5,6,DataTech Solutions,Display,All Ages,15 days,Instagram,0.07,"$9,716.00",4.36,New York,German,100,1643,1,Foodies,1/6/2021,Instagram_Foodies,0.08
6,7,NexGen Systems,Email,Women 35-44,60 days,Website,0.13,"$11,067.00",2.86,Los Angeles,Spanish,817,8749,10,Tech Enthusiasts,1/7/2021,Website_Tech Enthusiasts,0.08
7,8,DataTech Solutions,Search,Men 18-24,45 days,Google Ads,0.08,"$13,280.00",5.55,Los Angeles,Mandarin,624,7854,7,Outdoor Adventurers,1/8/2021,Google Ads_Outdoor Adventurers,0.08
8,9,Alpha Innovations,Social Media,Women 35-44,15 days,Facebook,0.09,"$18,066.00",6.73,Chicago,German,861,1754,6,Tech Enthusiasts,1/9/2021,Facebook_Tech Enthusiasts,0.06
9,10,TechCorp,Email,Women 35-44,15 days,Instagram,0.09,"$13,766.00",3.78,Los Angeles,English,642,3856,3,Tech Enthusiasts,1/10/2021,Instagram_Tech Enthusiasts,0.08


## Other Countings for CVR Benchmarks

In [194]:
# Absolute difference
mc_df_new['CVR_Diff'] = mc_df_new['Conversion_Rate'] - mc_df_new['Benchmark']

# Relative difference in % (how much percent above or below the market)
mc_df_new['Benchmark_Display'] = (mc_df_new['Benchmark'] * 100).round(2).astype(str) + '%'
mc_df_new['Actual_CVR_Display'] = (mc_df_new['Conversion_Rate'] * 100).round(2).astype(str) + '%'

mc_df_new['Performance_vs_Market_%'] = (((mc_df_new['Conversion_Rate'] / mc_df_new['Benchmark']) - 1) * 100).round(2).astype(str) + '%'

In [193]:
mc_df_new.head()

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,...,Impressions,Engagement_Score,Customer_Segment,Date,Join_Key,Benchmark,CVR_Diff,Benchmark_Display,Actual_CVR_Display,Performance_vs_Market_%
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,"$16,174.00",6.29,Chicago,...,1922,6,Health & Wellness,1/1/2021,Google Ads_Health & Wellness,0.07,-0.03,7.0%,4.0%,-42.86
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,"$11,566.00",5.61,New York,...,7523,7,Fashionistas,1/2/2021,Google Ads_Fashionistas,0.08,0.04,8.0%,12.0%,50.00
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,"$10,200.00",7.18,Los Angeles,...,7698,1,Outdoor Adventurers,1/3/2021,YouTube_Outdoor Adventurers,0.08,-0.01,8.0%,7.0%,-12.50
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,"$12,724.00",5.55,Miami,...,1820,7,Health & Wellness,1/4/2021,YouTube_Health & Wellness,0.08,0.03,8.0%,11.0%,37.50
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,"$16,452.00",6.50,Los Angeles,...,4201,3,Health & Wellness,1/5/2021,YouTube_Health & Wellness,0.08,-0.03,8.0%,5.0%,-37.50


In [195]:
mc_df_new["Acquisition_Cost"] = (
    mc_df_new["Acquisition_Cost"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)

mc_df_new["Acquisition_Cost"] = pd.to_numeric(mc_df_new["Acquisition_Cost"], errors="coerce")

In [ ]:
# mc_df_new.to_excel('marketing_benchmarks_report.xlsx', index=False)